In [129]:
import numpy as np 
import tensorflow as tf
import tensorflow_datasets as tfds

In [130]:
# Load dataset with info
mnist_dataset, mnist_info = tfds.load(
    name='mnist',
    with_info=True,
    as_supervised=True
)

In [131]:
# Separate train and test splits
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [132]:
# Compute sizes (Python ints, simpler than tf.cast)
num_validation_samples = int(0.1 * mnist_info.splits['train'].num_examples)  # 10% of training set
num_test_samples = int(mnist_info.splits['test'].num_examples)               # full test set

In [133]:
# Preprocessing function (scale pixels to [0, 1])
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

In [134]:
# Apply scaling
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [135]:
# Shuffle training data
BUFFER_SIZE = 10000
shuffled_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [136]:
# Split into validation and training
validation_data = shuffled_data.take(num_validation_samples).batch(num_validation_samples)
train_data = shuffled_data.skip(num_validation_samples).batch(100)
test_data = scaled_test_data.batch(100)

In [137]:
# Model parameters
hidden_layer_size = 100
output_size = 10

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [138]:
# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [139]:
# Train model
NUM_EPOCHS = 5
model.fit(
    train_data,
    epochs=NUM_EPOCHS,
    validation_data=validation_data,
    verbose=2
)

Epoch 1/5
540/540 - 7s - 13ms/step - accuracy: 0.9062 - loss: 0.3301 - val_accuracy: 0.9538 - val_loss: 0.1539
Epoch 2/5
540/540 - 4s - 7ms/step - accuracy: 0.9600 - loss: 0.1369 - val_accuracy: 0.9702 - val_loss: 0.1073
Epoch 3/5
540/540 - 3s - 6ms/step - accuracy: 0.9704 - loss: 0.0976 - val_accuracy: 0.9737 - val_loss: 0.0836
Epoch 4/5
540/540 - 3s - 6ms/step - accuracy: 0.9777 - loss: 0.0749 - val_accuracy: 0.9778 - val_loss: 0.0748
Epoch 5/5
540/540 - 3s - 6ms/step - accuracy: 0.9809 - loss: 0.0605 - val_accuracy: 0.9840 - val_loss: 0.0544


In [140]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {:.2f}. Test accuracy: {:.2f}%'.format(test_loss, test_accuracy * 100))

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9758 - loss: 0.0801
Test loss: 0.08. Test accuracy: 97.58%
